# import libraries

In [2]:
import pandas as pd
import numpy as np
import requests

# import Beautifulsoup,scrape info from wikipedia, store data in a dataframe

In [3]:
from bs4 import BeautifulSoup
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Data cleanup and reformat

Change inteter columns to strings 

In [7]:
df.columns=df.columns.astype(str)
print(df.columns)

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')


Change column names to assignment required ones

In [9]:
df.columns=['PostalCode','Borough','Neighborhood']


In [10]:
#drop first row of the original head from wiki site
df1=df.drop(df.index[0])
df1.head()

,PostalCode,Borough,Neighborhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [11]:
#remove rows that does not have Borough assigned
df2=df1[df1.Borough!='Not assigned']
df2.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [13]:
#next if Neighborhood is not assigned, it will need to have same as boroug
df2['Neighborhood'][df2.Neighborhood=='Not assigned']=df2['Borough']
print(df2.head())

  PostalCode           Borough      Neighborhood
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront
6        M5A  Downtown Toronto       Regent Park
7        M6A        North York  Lawrence Heights


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


If rows have same postcode, have rows combined in Neightborhood with comma

In [12]:
df3=df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
print(df3.shape)

(103, 3)


Check data layout

In [13]:
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Check number of rows and columns in final data frame df3

In [14]:
print(df3.shape)

(103, 3)


Sort on PostalCode so it is in order

In [15]:
y=df3.sort_values(by=['PostalCode'])

In [16]:
y.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Download and read the geo data into dataframe

In [17]:
gdf=pd.read_csv('http://cocl.us/Geospatial_data')
gdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Check the data rows

In [18]:
print(gdf.shape)

(103, 3)


Sort data on postal code

In [21]:
gdf.sort_values(by=['Postal Code'])
gdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Drop the PostalCode column and check effect

In [22]:
x=gdf.drop('Postal Code', axis=1)
x.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


Combine the geo data with the neighborhood data

In [23]:
result = pd.concat([y, x], axis=1)
result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Check dimention of combined data

In [24]:
print(result.shape)

(103, 5)


The final dataframe has 103 rows of postal code records and 5 columns as shown above